In [105]:
import pandas as pd
import numpy as np

In [23]:
test1_data = pd.read_csv("testfiles/data/test1.csv")

In [24]:
test1_data

,x1,x2,x3,x4,x5
0,NaN,-1.049925,-0.367390,-1.275708,0.642827
1,-1.365501,1.388988,0.949546,-0.013575,1.308596
2,2.222779,-0.956345,0.132482,-0.197584,-0.457551
3,1.223125,2.121199,0.362741,0.072131,NaN
4,0.751136,-0.787819,1.925311,NaN,1.329777
5,NaN,NaN,-0.287618,0.926628,NaN
6,1.026931,-0.675588,NaN,NaN,NaN
7,0.534427,0.683678,-0.860848,-1.240771,0.493723
8,NaN,0.490084,-0.728208,0.660641,NaN
9,0.480244,0.901406,1.329765,1.328672,-0.085088


## Test 1.1 Covariance Missing data, skip missing rows

In [32]:
test1_cov = test1_data.dropna().cov()

In [36]:
test_1_1_vals = pd.read_csv("testfiles/data/testout_1.1.csv")
error_epsilon = 1e-12
for i in range(len(test1_cov)):
    for j in range(len(test1_cov.columns)):
        assert abs(test1_cov.iloc[i,j] - test_1_1_vals.iloc[i,j]) < error_epsilon, f"row {i} and column {j} of the data do not match"

## Test 1.2 Correlation Missing data, skip missing rows

In [34]:
test1_corr = test1_data.dropna().corr()

In [35]:
test_1_2_vals = pd.read_csv("testfiles/data/testout_1.2.csv")
error_epsilon = 1e-12
for i in range(len(test1_corr)):
    for j in range(len(test1_corr.columns)):
        assert abs(test1_corr.iloc[i,j] - test_1_2_vals.iloc[i,j]) < error_epsilon, f"row {i} and column {j} of the data do not match"

## Test 1.3 Covariance Missing Data, Pairwise

In [37]:
test1_cov_missing = test1_data.cov()

In [40]:
test_1_3_vals = pd.read_csv("testfiles/data/testout_1.3.csv")
error_epsilon = 1e-12
for i in range(len(test1_cov_missing)):
    for j in range(len(test1_cov_missing.columns)):
        assert abs(test1_cov_missing.iloc[i,j] - test_1_3_vals.iloc[i,j]) < error_epsilon, f"row {i} and column {j} of the data do not match"

## Test 1.4 Correlation Missing Data, Pairwise

In [42]:
test1_corr_missing = test1_data.corr()

In [44]:
test_1_4_vals = pd.read_csv("testfiles/data/testout_1.4.csv")
error_epsilon = 1e-12
for i in range(len(test1_corr_missing)):
    for j in range(len(test1_corr_missing.columns)):
        assert abs(test1_corr_missing.iloc[i,j] - test_1_4_vals.iloc[i,j]) < error_epsilon, f"row {i} and column {j} of the data do not match"

## Test 2.1 EW Covariance, lambda=0.97

In [45]:
test2_data = pd.read_csv("testfiles/data/test2.csv")

In [83]:
lambda_= 0.97
ewm_cov = test2_data.ewm(alpha = (1-lambda_),).cov(bias=True)
last_ewm_cov_matrix = ewm_cov.loc[ewm_cov.index.get_level_values(0).max()]

In [86]:
ewm_cov

x1        x2        x3        x4        x5
0  x1  0.000000  0.000000  0.000000  0.000000  0.000000
   x2  0.000000  0.000000  0.000000  0.000000  0.000000
   x3  0.000000  0.000000  0.000000  0.000000  0.000000
   x4  0.000000  0.000000  0.000000  0.000000  0.000000
   x5  0.000000  0.000000  0.000000  0.000000  0.000000
...         ...       ...       ...       ...       ...
39 x1  0.848630  0.120043  0.179149  0.083942  0.055432
   x2  0.120043  1.079589  0.024682  0.115124 -0.429611
   x3  0.179149  0.024682  0.736457  0.133765  0.069033
   x4  0.083942  0.115124  0.133765  0.867932  0.112788
   x5  0.055432 -0.429611  0.069033  0.112788  1.137927

[200 rows x 5 columns]

In [84]:
last_ewm_cov_matrix

,x1,x2,x3,x4,x5
x1,0.848630,0.120043,0.179149,0.083942,0.055432
x2,0.120043,1.079589,0.024682,0.115124,-0.429611
x3,0.179149,0.024682,0.736457,0.133765,0.069033
x4,0.083942,0.115124,0.133765,0.867932,0.112788
x5,0.055432,-0.429611,0.069033,0.112788,1.137927


In [85]:
test_2_1_vals = pd.read_csv("testfiles/data/testout_2.1.csv")
error_epsilon = 1e-12
for i in range(len(last_ewm_cov_matrix)):
    for j in range(len(last_ewm_cov_matrix.columns)):
        assert abs(last_ewm_cov_matrix.iloc[i,j] - test_2_1_vals.iloc[i,j]) < error_epsilon, f"row {i} and column {j} of the data do not match"

## Test 2.2 EW Correlation, lambda=0.94

In [87]:
lambda_= 0.94
ewm_corr = test2_data.ewm(alpha = (1-lambda_),).corr()
last_ewm_corr_matrix = ewm_corr.loc[ewm_corr.index.get_level_values(0).max()]

In [88]:
last_ewm_corr_matrix

,x1,x2,x3,x4,x5
x1,1.000000,0.084787,0.190100,0.129045,0.070706
x2,0.084787,1.000000,-0.077432,0.202228,-0.442546
x3,0.190100,-0.077432,1.000000,0.198267,0.102093
x4,0.129045,0.202228,0.198267,1.000000,0.119541
x5,0.070706,-0.442546,0.102093,0.119541,1.000000


In [79]:
test_2_2_vals = pd.read_csv("testfiles/data/testout_2.2.csv")
error_epsilon = 1e-12
for i in range(len(last_ewm_corr_matrix)):
    for j in range(len(last_ewm_corr_matrix.columns)):
        assert abs(last_ewm_corr_matrix.iloc[i,j] - test_2_2_vals.iloc[i,j]) < error_epsilon, f"row {i} and column {j} of the data do not match"

## Test 2.3 Covariance with EW Variance (l=0.97), EW Correlation (l=0.94)

In [117]:
lambda_= 0.97
ewm_var = test2_data.ewm(alpha = (1-lambda_),).var(bias=True)
std_devs = np.sqrt(ewm_var.iloc[-1])
std_dev_products_matrix = np.outer(std_devs, std_devs)

lambda_= 0.94
ewm_corr = test2_data.ewm(alpha = (1-lambda_),).corr()
last_ewm_corr_matrix = ewm_corr.loc[ewm_corr.index.get_level_values(0).max()]

covariance_matrix = last_ewm_corr_matrix*std_dev_products_matrix

In [118]:
test_2_3_vals = pd.read_csv("testfiles/data/testout_2.3.csv")
error_epsilon = 1e-12
for i in range(len(covariance_matrix)):
    for j in range(len(covariance_matrix.columns)):
        assert abs(covariance_matrix.iloc[i,j] - test_2_3_vals.iloc[i,j]) < error_epsilon, f"row {i} and column {j} of the data do not match"

## Test 3.1 near_psd covariance

In [121]:
import pyfinance

ModuleNotFoundError: No module named 'numpy.lib.nanfunctions'

In [ ]:
data_3_1_vals = pd.read_csv("testfiles/data/testout_1.3.csv")

In [119]:
test_3_1_vals = pd.read_csv("testfiles/data/testout_3.1.csv")

## Test 3.3 Higham Covariance

## Test 6.1 Calculate Arithmetic Returns

In [144]:
test_6_data = pd.read_csv("testfiles/data/test6.csv")
test_6_data = test_6_data.set_index("Date")

In [149]:
arithmetic_returns = test_6_data.pct_change().dropna()

In [153]:
test_6_1_vals = pd.read_csv("testfiles/data/testout6_1.csv")
test_6_1_vals = test_6_1_vals.set_index("Date")
error_epsilon = 1e-12

for i in range(len(arithmetic_returns)):
    for j in range(len(arithmetic_returns.columns)):
        assert abs(arithmetic_returns.iloc[i,j] - test_6_1_vals.iloc[i,j]) < error_epsilon, f"row {i} and column {j} of the data do not match"

## Test 6.2 Calculate Log Returns

In [167]:
log_returns = np.log(test_6_data / test_6_data.shift(1)) # log(Pt / Pt-1)
log_returns = log_returns.dropna()

In [168]:
test_6_2_vals = pd.read_csv("testfiles/data/testout6_2.csv")
test_6_2_vals = test_6_2_vals.set_index("Date")
error_epsilon = 1e-12

for i in range(len(arithmetic_returns)):
    for j in range(len(arithmetic_returns.columns)):
        assert abs(log_returns.iloc[i,j] - test_6_2_vals.iloc[i,j]) < error_epsilon, f"row {i} and column {j} of the data do not match"